In [2]:
import numpy as np
import cv2
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import glob
import pickle



# prepare object points
nx = 9#number of inside corners in x
ny = 6#number of inside corners in y

images = glob.glob('camera_cal/calibration*.jpg')

objPoints = []
imgPoints = []


objP = np.zeros((6*9,3),np.float32)
objP[:,:2] = np.mgrid[0:9,0:6].T.reshape(-1,2)


for idx, image in enumerate(images) :  
    img = mpimg.imread(image)
    grayscale = cv2.cvtColor(img , cv2.COLOR_BGR2GRAY)
    #chess board corners
    ret, corners = cv2.findChessboardCorners(grayscale , (nx,ny), None)
    if ret == True :
        cv2.drawChessboardCorners(img , (nx,ny), corners , ret)
        imgPoints.append(corners)
        objPoints.append(objP)
ret, mtx, dist, rvecs, tvecs = cv2.calibrateCamera(objPoints, imgPoints, grayscale.shape[::-1], None, None)
undist=cv2.undistort(img, mtx, dist, None, mtx)


dist_pickle = {}
dist_pickle["mtx"]=mtx
dist_pickle["dist"]=dist
pickle.dump(dist_pickle,open("camera_cal/calibration_pickle.p","wb"))

In [6]:
def undist(image):
    dist_pickle = pickle.load(open("camera_cal/calibration_pickle.p","rb"))
    mtx = dist_pickle["mtx"]
    dist = dist_pickle["dist"]    
    result = cv2.undistort(image, mtx, dist, None, mtx)
    return result


images = glob.glob('camera_cal/calibration*.jpg')
for idx,fname in enumerate(images):
    img = cv2.imread(fname)
    img = undist(img)
    writename = 'output_images/Camera_cal/chessboard'+str(idx)+'.jpg'
    cv2.imwrite(writename,img)

    
    
